In [1]:
# !pip install sqlalchemy
# !pip install psycopg2
# !pip install psycopg2-binary

     |████████████████████████████████| 2.9MB 4.9MB/s 


In [0]:
# Import the SQL ALchemy engine
from sqlalchemy import create_engine, inspect
import pandas as pd

In [0]:
# Database credentials
postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'dvdrentals' #database_name

# use the credentials to start a connection
engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(
    postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db))

In [4]:
# inspect table_names & corresponding column_names in dvdrentals database
inspector = inspect(engine)

for table_name in inspector.get_table_names():
  print('\nTable: ' + table_name)

  column_list = []
  for column in inspector.get_columns(table_name):
       column_list.append(column['name'])
  print('Columns: {}'.format(column_list))


Table: actor
Columns: ['actor_id', 'first_name', 'last_name', 'last_update']

Table: address
Columns: ['address_id', 'address', 'address2', 'district', 'city_id', 'postal_code', 'phone', 'last_update']

Table: category
Columns: ['category_id', 'name', 'last_update']

Table: city
Columns: ['city_id', 'city', 'country_id', 'last_update']

Table: country
Columns: ['country_id', 'country', 'last_update']

Table: customer
Columns: ['customer_id', 'store_id', 'first_name', 'last_name', 'email', 'address_id', 'activebool', 'create_date', 'last_update', 'active']

Table: film_actor
Columns: ['actor_id', 'film_id', 'last_update']

Table: film_category
Columns: ['film_id', 'category_id', 'last_update']

Table: inventory
Columns: ['inventory_id', 'film_id', 'store_id', 'last_update']

Table: language
Columns: ['language_id', 'name', 'last_update']

Table: Example
Columns: ['id', 'name']

Table: MYDATA
Columns: ['station', 'name', 'latitude', 'longitude', 'elevation', 'date', 'prcp']

Table: film

# DRILLS
## 1. How many movies are released for each rating?

In [5]:
# SQL statement
sql = '''
SELECT DISTINCT rating, COUNT(*) AS movie_count
FROM film
GROUP BY rating
'''

# Execute the SQL statement
movies = engine.execute(sql)

# dispose the connection
engine.dispose()

# use fetchall() to get a list of rows from the results.
rows = movies.fetchall()

# convert to df
rows_df = pd.DataFrame(rows, columns = movies.keys())
rows_df

,rating,movie_count
0,G,178
1,PG,194
2,PG-13,223
3,R,195
4,NC-17,210


## 2. What is the average rental duration for each rating?

In [6]:
# SQL statement
sql = '''
SELECT DISTINCT rating, ROUND(AVG(rental_duration), 2) AS avg_rental_duration
FROM film
GROUP BY rating;
'''

# Execute the SQL statement
movies = engine.execute(sql)

# dispose the connection
engine.dispose()

# use fetchall() to get a list of rows from the results.
rows = movies.fetchall()

# convert to df
rows_df = pd.DataFrame(rows, columns = movies.keys())
rows_df

,rating,avg_rental_duration
0,G,4.84
1,PG,5.08
2,PG-13,5.05
3,R,4.77
4,NC-17,5.14


## 3. What is the mean movie length? Calculate this by defining a function.

In [7]:
def average(num_list):
  total = 0
  for i in num_list:
    total += i

  avg = total/len(num_list)
  return avg

# test
average([2,4,6])

4.0

In [8]:
# SQL statement
sql = '''
SELECT length
FROM film;
'''

# Execute the SQL statement
length = engine.execute(sql)

# dispose the connection
engine.dispose()

# use fetchall() to get a list of rows from the results.
rows = length.fetchall()
type(rows) #RowProxy

# extract numeric value from RowProxy objects into a list of numbers
length = [x['length'] for x in rows]

print('Mean movie length: {}'.format(average(length)))

Mean movie length: 115.272


In [9]:
# double check using sql
sql = '''
SELECT ROUND(AVG(length), 3)
FROM film;
'''

# Execute the SQL statement
length = engine.execute(sql)

# dispose the connection
engine.dispose()

rows = length.fetchall()
rows[0][0]

Decimal('115.272')

## 4. What is the median movie length? Calculate this by defining a function.

In [10]:
def median(num_list):
  num_list.sort()
  len_list = len(num_list)

  if len_list % 2 == 0:
    mid_index = int(len_list/2 - 1)
    result = (num_list[mid_index] + num_list[mid_index + 1])/2

  else:
    mid_index = int(round(len_list/2) - 1)
    result = num_list[mid_index]

  return result

# test
median([4,8,2,6]) #5
median([4,2,6]) #4

4

In [11]:
# SQL statement
sql = '''
SELECT length
FROM film;
'''

# Execute the SQL statement
length = engine.execute(sql)

# dispose the connection
engine.dispose()

# use fetchall() to get a list of rows from the results.
rows = length.fetchall()

# extract numeric value from RowProxy objects into a list of numbers
length = [x['length'] for x in rows]

print('Median movie length: {}'.format(median(length)))

Median movie length: 114.0


## 5. Calculate the standard deviation of the movie lengths. Calculate this by defining a function.
#### Standard Deviation
$$ \sigma  = \sqrt\frac{\sum(X - \bar X)^2}{N - 1} $$
Where 
 * $X$ - Each individual value
 * $\bar X$ - The mean of all values
 * $ \sum$ - The sum of whatever comes next
 * $ N $ - The total number of values

In [12]:
def std_dev(num_list):
  mean = average(num_list)
  len_list = len(num_list)

  sum_of_square_of_difference = 0
  for i in num_list:
    square_of_diff = (i - mean)**2
    sum_of_square_of_difference += square_of_diff

  numerator = sum_of_square_of_difference
  denominator = len_list - 1

  return round((numerator/denominator)**0.5, 3)

std_dev([4,5,6])

1.0

In [13]:
# SQL statement
sql = '''
SELECT length
FROM film;
'''

# Execute the SQL statement
length = engine.execute(sql)

# dispose the connection
engine.dispose()

# use fetchall() to get a list of rows from the results.
rows = length.fetchall()

# extract numeric value from RowProxy objects into a list of numbers
length = [x['length'] for x in rows]

print('Standard deviation of movie length: {}'.format(std_dev(length)))

Standard deviation of movie length: 40.426


In [14]:
# double check using sql
sql = '''
SELECT ROUND(STDDEV(length), 3)
FROM film;
'''

# Execute the SQL statement
length = engine.execute(sql)

# dispose the connection
engine.dispose()

rows = length.fetchall()
rows[0][0]

Decimal('40.426')